<h1>CNN Model</h1>

In [1]:
from pycocotools.coco import COCO
import matplotlib
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as functions
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
import re
from scipy import stats

In [2]:
DATADIR = "cocodoom/"

dataSplit, run = "run-full-train", "run1"

annFile = '{}{}.json'.format(DATADIR,dataSplit)

In [3]:
coco = COCO(annFile)

loading annotations into memory...
Done (t=19.99s)
creating index...
index created!


In [4]:
player_positions = {}

player_regex = re.compile(r"^(?P<tic>\d+)\s+player:(?P<x>-?\d+\.\d+),(?P<y>-?\d+\.\d+),(?P<z>-?\d+\.\d+),(?P<angle>-?\d+\.\d+)$")

with open(DATADIR+run+"/log.txt", 'r') as log_file:
    for line in log_file:
        if "player" in line:
            line = line.strip()
            tic, pos = line.split("player:")
            x, y, z, angle = pos.split(",")

            # Store position in the dictionary
            player_positions[int(tic)] = (float(x), float(y), float(z), float(angle))

In [5]:
class DoomMotionDataset(Dataset):
    def __init__(self, transform=None):
        self.img_ids = coco.getImgIds()
        self.transform = transform

    def __len__(self):
        return len(self.img_ids)

    def getSegmentationMask(self, rgb_filename):
        return rgb_filename.replace("rgb", "objects")

    def getDepthMask(self, rgb_filename):
        return rgb_filename.replace("rgb", "depth")

    def color_to_index(self, segmentation_image):
        # Map colors to class indices
        r, g, b = segmentation_image
        pixel_values = r + (g *  2**8) + (b * 2**16)  # From cocodoom documentation, converts to an object id

        class_map = torch.full_like(pixel_values, 3, dtype=torch.long)

        sky = (1 << 23) + 0
        horizontal = (1 << 23) + 1
        vertical = (1 << 23) + 2
        
        class_map[x == sky] = 0
        class_map[x == horizontal] = 1
        class_map[x == vertical] = 2
        return class_map

    def load_image(self, path):
        img = Image.open(path)
        return transforms.ToTensor()(img)

    def __getitem__(self, idx):
        # Load the RGB image
        rgb_filename = coco.loadImgs(self.img_ids[idx])[0]['file_name']
        tic = int(rgb_filename.replace(".png", "").split("/")[-1])
        next_tic = tic+1

        player_position = player_positions[tic]
        if next_tic not in player_positions:
            next_player_position = player_position
        else:
            next_player_position = player_positions[next_tic]
        dx = next_player_position[0] - player_position[0]
        dy = next_player_position[1] - player_position[1]
        dz = next_player_position[2] - player_position[2]
        dangle = next_player_position[3] - player_position[3]
        dangle = (dangle + np.pi) % 2*np.pi - np.pi
        motion_vector = (dx, dy, dz, dangle)
            
        
        # Load and process the segmentation map
        seg_image = self.load_image(self.getSegmentationMask(DATADIR + rgb_filename))
        seg_class_map = self.color_to_index(seg_image)
        seg_class_one_hot = functions.one_hot(seg_class_map, num_classes=4).to(dtype=torch.float).permute(2, 0, 1)
        #print(seg_class_one_hot.shape)
        #seg_class_mode = self.mode_pooling(seg_class_one_hot, 2)
        #print(seg_class_mode.shape)

        # Load depth map
        depth_mask = self.load_image(self.getDepthMask(DATADIR + rgb_filename))
        #depth_mask_mode = self.mode_pooling(depth_mask, 2)
        #print(depth_mask.shape)

        # Combine depth and segmentation as separate channels
        combined = torch.cat([seg_class_one_hot, depth_mask], dim=0)
        
        # Motion vector (label)
        motion_vector = torch.tensor(motion_vector, dtype=torch.float32)
        
        return combined, motion_vector


In [18]:
class NeuralNetwork(nn.Module):
  def __init__(self, activation_function=functions.relu, device=torch.device("cpu")):
    super(NeuralNetwork, self).__init__()

    # Max pooling like this should preserve all of the present classes in a 2x2 window into one vector.
    self.pool = nn.MaxPool2d(kernel_size=(2, 2)).to(device)
      
    self.activation_function = activation_function
    self.conv1 = nn.Conv2d(in_channels=5, out_channels=8, kernel_size=(5, 5), padding=0, bias=False).to(device)
    self.bn1 = nn.BatchNorm2d(num_features=8).to(device)

    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), padding=2, bias=False).to(device)
    self.bn2 = nn.BatchNorm2d(num_features=16).to(device)

    self.flatten = nn.Flatten().to(device)
      
    self.fc1 = nn.Linear(16000, 456).to(device)
    self.bn3 = nn.BatchNorm1d(num_features=456).to(device)

    self.out = nn.Linear(456, 4).to(device)

  def forward(self, x):
    x = self.pool(x)
    x = self.activation_function(self.bn1(self.conv1(x)))
    x = self.pool(x)
    x = self.activation_function(self.bn2(self.conv2(x)))
    x = self.pool(x)
    x = self.flatten(x)
    x = self.activation_function(self.bn3(self.fc1(x)))
    x = self.out(x)
    return x

In [19]:
batch_size = 256
learning_rate = 1e-3
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetwork(device=device).to(device)

train_dataset = DoomMotionDataset()
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

# Loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")
    for batch_idx, (inputs, targets) in enumerate(progress_bar):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
    
        loss = criterion(outputs, targets)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        progress_bar.set_postfix({
            "batch_loss": loss.item(),
            "batch_index": batch_idx + 1,
            "batch_size": inputs.size(0)
        })

    # Average loss per epoch
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), "image_cnn.pth")

cuda



poch 1/10: 100%|█████████████| 991/991 [1:46:49<00:00,  6.47s/batch, batch_loss=33.7, batch_index=991, batch_size=220]

Epoch 1, Loss: 491.6127



poch 2/10: 100%|█████████████| 991/991 [1:51:46<00:00,  6.77s/batch, batch_loss=32.9, batch_index=991, batch_size=220]

Epoch 2, Loss: 491.3679


...

Epoch 8, Loss: 489.9004



poch 9/10: 100%|█████████████| 991/991 [2:16:08<00:00,  8.24s/batch, batch_loss=28.2, batch_index=991, batch_size=220]

Epoch 9, Loss: 488.8508



poch 10/10: 100%|████████████| 991/991 [2:42:03<00:00,  9.81s/batch, batch_loss=25.1, batch_index=991, batch_size=220]

Epoch 10, Loss: 487.5924
